In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from config import gkey
# API Key for gmap "gkey" is contained in config.py
import gmaps
import gmaps.geojson_geometries
from matplotlib.cm import viridis, inferno, plasma
from matplotlib.colors import to_hex
gmaps.configure(api_key= gkey)

In [3]:
rds_connection_string = "postgres:postgres@localhost:5432/COVID_df"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [4]:
happy_df = pd.read_sql_query('select * from happiness_table', con=engine)

In [5]:
happy_df.head()

,country,ranking,score,social_support,freedom_index,corruption_index
0,Finland,1,7.769,1.587,0.596,0.393
1,Denmark,2,7.600,1.573,0.592,0.410
2,Norway,3,7.554,1.582,0.603,0.341
3,Iceland,4,7.494,1.624,0.591,0.118
4,Netherlands,5,7.488,1.522,0.557,0.298


In [6]:
covid_df = pd.read_sql_query('select * from covid_cases', con=engine)

In [7]:
covid_df.head()

,country_region,population,totalcases,totaldeaths,totaltests
0,USA,330854064.0,1881205.0,108059.0,18603174.0
1,Brazil,212442762.0,558237.0,31309.0,930013.0
2,Russia,145929848.0,423741.0,5037.0,11151622.0
3,Spain,46753443.0,287012.0,27127.0,4063843.0
4,UK,67858826.0,277985.0,39369.0,4615146.0


In [8]:
country_df = pd.read_sql_query('select * from countrycode_table', con=engine)

In [9]:
country_df.head()

,iso_a3,iso_a2,country,continent,subregion,country_code,continent_code,subregion_code
0,AFG,AF,Afghanistan,Asia,Southern Asia,4,142.0,34.0
1,ALA,AX,Åland Islands,Europe,Northern Europe,248,150.0,154.0
2,ALB,AL,Albania,Europe,Southern Europe,8,150.0,39.0
3,DZA,DZ,Algeria,Africa,Northern Africa,12,2.0,15.0
4,ASM,AS,American Samoa,Oceania,Polynesia,16,9.0,61.0


In [10]:
merged_df = pd.merge(happy_df, country_df, how='inner', left_on='country', right_on='country')

In [11]:
merged_df.set_index("country", inplace=True)
merged_df.head()

,ranking,score,social_support,freedom_index,corruption_index,iso_a3,iso_a2,continent,subregion,country_code,continent_code,subregion_code
country,,,,,,,,,,,,
Finland,1,7.769,1.587,0.596,0.393,FIN,FI,Europe,Northern Europe,246,150.0,154.0
Denmark,2,7.600,1.573,0.592,0.410,DNK,DK,Europe,Northern Europe,208,150.0,154.0
Norway,3,7.554,1.582,0.603,0.341,NOR,NO,Europe,Northern Europe,578,150.0,154.0
Iceland,4,7.494,1.624,0.591,0.118,ISL,IS,Europe,Northern Europe,352,150.0,154.0
Netherlands,5,7.488,1.522,0.557,0.298,NLD,NL,Europe,Western Europe,528,150.0,155.0


In [12]:
merged_df.to_csv("./CSV/happiness_w_countrycode.csv")

### Save to DB update 3

In [13]:
merged_df.to_sql(name='happiness_w_countrycode_table', con=engine, if_exists='append', index=True, index_label="country")

### Add GeoJSON layer

In [14]:
countries_geojson = gmaps.geojson_geometries.load_geometry('countries')

In [15]:
def calculate_color(input_data):
    # input_data is between 0 and 1
    # invert gini so that high inequality gives dark color
    inverse_data = 1.0 - input_data

    # transform the gini coefficient to a matplotlib color
#     mpl_color = viridis(inverse_data)
#     mpl_color = plasma(inverse_data)
    mpl_color = inferno(inverse_data)
    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [16]:
country_names = merged_df.index
country_codes = list(merged_df["iso_a3"])

In [17]:
# Assign different color per freedom_index of each country. Some country in merged_df does not matched with country name in gmaps.geojson_geometries.
# Therefore it is checked again using ISO_A3

colors=[]
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    country_code = feature['properties']['ISO_A3']
    try:
        if country_name in country_names:
            input_value = merged_df.loc[merged_df.index==country_name]["freedom_index"][0]
        elif country_code in country_codes:
            input_value = merged_df.loc[merged_df.iso_a3==country_code]["freedom_index"][0]
        else: input_value = 0
        color = calculate_color(input_value)
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [18]:
fig = gmaps.figure(zoom_level = 3, center=(40.00, 40.00), layout={
        'width': '1000px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
gini_layer = gmaps.geojson_layer(
    countries_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='1000px'))

### Using Bins and color

In [19]:
merged_df.ranking.max()

156

In [20]:
# Grouping countries per ranking bins and assign different color label for each bin.
# Color code refer to the color chart from http://my.crazyartzone.com/dmc.asp
bins = [1, 11, 21, 31, 41, 51, 156]
color_code_label = [0,1,2,3,4,5]
color_code = [(191/255,45/255,45/255,1.0),(227/255,109/255,109/255,1.0),(241/255,135/255,135/255,1.0),(245/255,173/255,173/255,1.0),(1.0, 201/255, 201/255,1.0),(1.0,226/255,226/255,1.0)]
merged_df["color_code"] = pd.cut(merged_df["ranking"], bins, labels=color_code_label, include_lowest=True)

In [21]:
merged_df.head()

,ranking,score,social_support,freedom_index,corruption_index,iso_a3,iso_a2,continent,subregion,country_code,continent_code,subregion_code,color_code
country,,,,,,,,,,,,,
Finland,1,7.769,1.587,0.596,0.393,FIN,FI,Europe,Northern Europe,246,150.0,154.0,0
Denmark,2,7.600,1.573,0.592,0.410,DNK,DK,Europe,Northern Europe,208,150.0,154.0,0
Norway,3,7.554,1.582,0.603,0.341,NOR,NO,Europe,Northern Europe,578,150.0,154.0,0
Iceland,4,7.494,1.624,0.591,0.118,ISL,IS,Europe,Northern Europe,352,150.0,154.0,0
Netherlands,5,7.488,1.522,0.557,0.298,NLD,NL,Europe,Western Europe,528,150.0,155.0,0


In [22]:
colors=[]
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    country_code = feature['properties']['ISO_A3']
    try:
        if country_name in country_names:
            input_value = merged_df.loc[merged_df.index==country_name]["color_code"][0]
        elif country_code in country_codes:
            input_value = merged_df.loc[merged_df.iso_a3==country_code]["color_code"][0]
        else: input_value = 5
        color = to_hex(color_code[input_value], keep_alpha=False)
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [23]:
fig = gmaps.figure(zoom_level = 3, center=(40.00, 40.00), layout={
        'width': '1000px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})

In [24]:
gini_layer = gmaps.geojson_layer(
    countries_geojson,
    fill_color=colors,
    stroke_color=None,
    fill_opacity=0.9)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='1000px'))

In [25]:
capital_df = pd.read_sql_query('select * from capital_table', con=engine)
capital_df.head()

,country,capital,lat,lng,iso_a2
0,Somaliland,Hargeisa,9.550000,44.050000,None
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF
3,Palestine,Jerusalem,31.766667,35.233333,PS
4,Aland Islands,Mariehamn,60.116667,19.900000,AX


In [26]:
country_list = list(capital_df.country)
iso_a2_list = list(capital_df.iso_a2)
lat_list = []
lng_list = []
count = 0
# merged_df["lat"]=[]
# merged_df["lng"]=[]
for index, row in merged_df.iterrows():
    country_name = index
    if country_name in country_list:
        lat_data = capital_df.loc[capital_df.country==country_name]["lat"]
        lng_data = capital_df.loc[capital_df.country==country_name]["lng"]
        lat_list.append(list(lat_data)[0])
        lng_list.append(list(lng_data)[0])
    elif row.iso_a2 in iso_a2_list:
        lat_data = capital_df.loc[capital_df.iso_a2==row.iso_a2]["lat"]
        lng_data = capital_df.loc[capital_df.iso_a2==row.iso_a2]["lng"]
        lat_list.append(list(lat_data)[0])
        lng_list.append(list(lng_data)[0])
    else:
        lat_data = ""
        lng_data = ""
        lat_list.append(lat_data)
        lng_list.append(lng_data)
        count = 1+count
print(f"number of country mismatching is {count}")
merged_df["lat"] = lat_list
merged_df["lng"] = lng_list
merged_df.head()


number of country mismatching is 0


,ranking,score,social_support,freedom_index,corruption_index,iso_a3,iso_a2,continent,subregion,country_code,continent_code,subregion_code,color_code,lat,lng
country,,,,,,,,,,,,,,,
Finland,1,7.769,1.587,0.596,0.393,FIN,FI,Europe,Northern Europe,246,150.0,154.0,0,60.166667,24.933333
Denmark,2,7.600,1.573,0.592,0.410,DNK,DK,Europe,Northern Europe,208,150.0,154.0,0,55.666667,12.583333
Norway,3,7.554,1.582,0.603,0.341,NOR,NO,Europe,Northern Europe,578,150.0,154.0,0,59.916667,10.750000
Iceland,4,7.494,1.624,0.591,0.118,ISL,IS,Europe,Northern Europe,352,150.0,154.0,0,64.150000,-21.950000
Netherlands,5,7.488,1.522,0.557,0.298,NLD,NL,Europe,Western Europe,528,150.0,155.0,0,52.350000,4.916667


In [27]:
data_df = merged_df[["ranking", "score", "iso_a3"]]

In [28]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Country_code</dt><dd>{iso_a3}</dd>
<dt>Ranking</dt><dd>{ranking}</dd>
<dt>Score</dt><dd>{score}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
country_info = [info_box_template.format(**row) for index, row in data_df.iterrows()]
locations = merged_df[["lat", "lng"]]

In [29]:
# Add marker layer ontop
marker_layer = gmaps.marker_layer(locations, hover_text=merged_df.index, info_box_content=country_info)
fig.add_layer(marker_layer)

# Display figure


In [30]:
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='1000px'))